Import Torch Packages

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import torch.optim as optim

#### Import Gym Packages

In [2]:
from ale_py import ALEInterface
ale = ALEInterface()

A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]


In [3]:
from ale_py.roms import Breakout
ale.loadROM(Breakout)

Game console created:
  ROM file:  /home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/AutoROM/roms/breakout.bin
  Cart Name: Breakout - Breakaway IV (1978) (Atari)
  Cart MD5:  f34f08e5eb96e500e851a80be3277a56
  Display Format:  AUTO-DETECT ==> NTSC
  ROM Size:        2048
  Bankswitch Type: AUTO-DETECT ==> 2K

Running ROM file...
Random seed is 1647972765


In [4]:
import gym
from gym.wrappers import FrameStack
from gym.wrappers.monitoring.video_recorder import VideoRecorder

#### All Other Packages

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import trange
import random
import copy
from loguru import logger
import wandb
import math

/home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [6]:
wandb.login(key="5966d774f384473f3d7ed674ef762b1a26a54d63")

wandb: Currently logged in as: beegass (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/beegass/.netrc


True

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # check if GPU is available

##### The Meat And Potatoes 

In [8]:
class Gym_Env():
    def __init__(self, env_name, max_steps=1000, max_episodes=10000):
        self.le_env = FrameStack(gym.make(env_name, render_mode='rgb_array'), 4)
        self.max_steps = max_steps
        self.max_episodes = max_episodes
        

In [9]:
class Replay_Buffer():
    def __init__(self, capacity, mini_batch_size=128):
        self.rb = []
        self.capacity = capacity
        self.mini_batch_size = mini_batch_size
        self.current_batch = None

    def get_rb_batch(self):
        sample = random.sample(self.rb, self.mini_batch_size)
        states, actions, next_states, rewards, done = zip(*sample[0: (self.mini_batch_size - 1)])
        return states, actions, next_states, rewards, done
    
    def add_to_rb(self, new_transition):
        if len(self.rb) >= self.capacity:
            del self.rb[0] 
        self.rb.append(new_transition)

In [10]:
# ripped from https://nn.labml.ai/rl/dqn/index.html
class Deep_Q_Network(nn.Module):
    def __init__(self):
        super(Deep_Q_Network, self).__init__()
        
        self.network = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.lin = nn.Linear(in_features=7 * 7 * 64, out_features=512)
        self.activation = nn.ReLU()
        
        self.state_value = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=1)
            )
        
        self.action_value = nn.Sequential(
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=4),
        )
    
    def forward(self, x):
        x = self.network(x)
        x = x.reshape((-1, 7 * 7 * 64))
        x = self.activation(self.lin(x))
        action_value = self.action_value(x)
        state_value = self.state_value(x)
        action_score_centered = action_value - action_value.mean(dim=-1, keepdim=True)
        q = state_value + action_score_centered
        return q  
        
    

In [11]:
class Agent(nn.Module):
    def __init__(self, pred_model):
        super(Agent, self).__init__()
        self.agent = pred_model
        self.target = None
        
    def agent_policy(self, state, pred_model=True, grad=False):
        q_val = -1
        state = state.to(device)
        if pred_model:
            if grad:
                q_val = self.agent(state)
            else:
                with torch.no_grad():
                    q_val = self.agent(state)
        else:
            with torch.no_grad():
                q_val = self.target(state)
        return q_val
    
    def copy_pred_to_target(self):
        self.target = copy.deepcopy(self.agent)
    

##### Epsilon

In [12]:
def epsilon_greedy(environment, the_agent, state, epsilon=0.1):
    prob = random.random()
    action = environment.le_env.action_space.sample() # pick action from action space
    if prob < 1 - epsilon:
        q_val = the_agent.agent_policy(state) # retrieve best action, based off its action-value action
        action = torch.argmax(q_val)
    return action

In [13]:
def epsilon_decay(environment, episode_num, p_init=0.7, p_end=0.1):
    episode_num += 1
    current_episode_rate = (environment.max_episodes - episode_num) / environment.max_episodes
    epsilon_decay_rate = max(current_episode_rate, 0)
    return ((p_init - p_end) * (epsilon_decay_rate) + p_end)

##### Preprocess

In [14]:
def preprocess(state):
    
    # convert state to numpy array and then to torch tensor
    frame = torch.from_numpy(np.array(state).astype(np.float32))
    
    # reshape so that grayscaling is possible
    reshaped_frame = frame.reshape(4, 3, 210, 160)
    
    # grayscale image
    gray_frame = T.Grayscale()(reshaped_frame)
    
    # reshape image so network can process it
    reshaped_gray_frame = gray_frame.reshape(1, 4, 210, 160)
    
    # downscale image to 84x84
    small_gray_frame = T.Resize((84, 84))(reshaped_gray_frame)
    
    return small_gray_frame
    

##### Global Variables

In [15]:
def config_tune() -> None:
    cfg = {
        'method': 'bayes', #grid, random
        'metric': {
            'name': 'mean_reward',
            'goal': 'maximize'   
        },
        'parameters': {
            'alpha': {'distribution': 'uniform',
                                      'min': 0.000001,
                                      'max': 1},
            'gamma': {'distribution': 'uniform',
                                      'min': 0.1,
                                      'max': 1},
            'epsilon': {'distribution': 'uniform',
                                      'min': 0.90,
                                      'max': 1},
            'decay_rate': {'distribution': 'uniform',
                                      'min': 0.95,
                                      'max': 1},
            'batch_size': {
                'values': [16, 32, 64, 128]
            },
            'replay_buffer_size': {
                'value': 10000
            },
            'max_episodes': {
                'value': 200 
            },
            'max_steps': {
                'value': 10000
            }
        }
    }
    
    return cfg

In [16]:
def vanilla_config() -> None:
    cfg = {
        'alpha': 0.95,
        'gamma': 0.23,
        'epsilon': 0.95,
        'batch_size': 32,
        'replay_buffer_size': 5000,
        'max_episodes': 10000,
        'max_steps': 10000,
    }
    
    return cfg

##### Buidling Optimizers and Schedulers

In [17]:
def build_optimizer(model, optimizer_name='adam', learning_rate=0.01, weight_decay=0.01, momentum=0.9):
    try:
        optimizer = None
        if optimizer_name == "sgd":
            optimizer = optim.SGD(model.parameters(), 
                                  lr=learning_rate, 
                                  momentum=momentum)
            
        elif optimizer_name == "adam":
            optimizer = optim.Adam(model.parameters(), 
                                   lr=learning_rate, 
                                   weight_decay=weight_decay)
               
        return optimizer
    except:
        print("Error: Invalid optimizer specified.")
        sys.exit(1)

In [18]:
def build_scheduler(optimizer, sched_name='reduce_lr', patience=5, verbose=True):
    try: 
        sched = None
        if sched_name == "reduce_lr":
            sched = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                         mode='min', 
                                                         patience=patience, 
                                                         verbose=verbose)
        elif sched_name == 'TODO':
            pass
            #TODO: add other scheduler
            
        return sched
    except:
        logger.error("Error: Invalid scheduler specified.")
        sys.exit(1)

##### Training Loop

In [19]:
def train(replay_buffer, the_agent, loss_fn, optimizer, scheduler, gamma=0.95):
    mini_batch = replay_buffer.sample_rb()
    
    #1. copy weights from prediction network to target target network
    the_agent.copy_pred_to_target() 
    
    #3. retrieve (s, a, r, s') from mini_batch
    for transition in mini_batch:
        (state, _, next_state, reward, done) = transition
        next_state = next_state.detach().clone()
        y_j        = torch.tensor(reward, dtype = torch.float, device = device)
        
        _, pred_highest_q_val = the_agent.agent_policy(state, grad=True)
        if not done:   
            _, target_highest_q_val = the_agent.agent_policy(next_state, pred_model=False)
            y_j += gamma * target_highest_q_val
        
        output = loss_fn(pred_highest_q_val, y_j.detach())
        optimizer.zero_grad()
        output.backward()
        optimizer.step() 

In [20]:
def train2(replay_buffer, the_agent, loss_fn, optimizer, scheduler, gamma=0.95):
    #1. copy weights from prediction network to target target network
    the_agent.copy_pred_to_target() 
    
    #2. retrieve (s, a, r, s') from mini_batch
    states, actions, next_states, rewards, done = replay_buffer.get_rb_batch()
    
    states = torch.vstack(states)
    actions = torch.tensor(actions, dtype = torch.long, device = device).detach()
    next_states = torch.vstack(next_states)
    rewards = torch.tensor(rewards, dtype = torch.float, device = device)
    done = torch.tensor(done, dtype=torch.int32, device = device)
    
    pred_q_val_matrix = the_agent.agent_policy(states, grad=True)
    pred_q_val = torch.gather(pred_q_val_matrix, 1, actions.unsqueeze(1)).squeeze(1)
    target_q_val_matrix = the_agent.agent_policy(next_states, pred_model=False)
    target_q_val = torch.max(target_q_val_matrix, dim=1)[0]
    #zero_or_one = torch.ones(done.shape, dtype=torch.float, device = device) - done
    y_j = (rewards + (gamma * target_q_val)) * (1 - done)
    
    output = loss_fn(pred_q_val, y_j.detach())
    optimizer.zero_grad()
    output.backward()
    optimizer.step()

##### Episode Specific Loop

In [21]:
def run_episode(video, render, environment, the_agent, replay_buffer, state, epsilon, gamma, loss_fn, optimizer, scheduler):
    step_count = 0
    cumulative_reward = 0
    while True:
        
        #0. either explore or exploit
        action = epsilon_greedy(environment=environment,
                                the_agent=the_agent,
                                state=state,
                                epsilon=epsilon)
        
        #1. render environment only when flag is set to True
        if render:
            video.capture_frame()
        
        #2. pass action to environment
        (next_state, reward, done, metadata) = environment.le_env.step(action)
        
        #3. get s' back from environment and preprocess (s' -> preprocessed_s')
        preprocessed_next_state = preprocess(next_state)
        
        #4. add transition (s, a, s', r) to replay buffer
        replay_buffer.add_to_rb((state, action, preprocessed_next_state, reward, done))
        
        #5. if replay buffer is full, sample mini batch and update model
        if len(replay_buffer.rb) > replay_buffer.mini_batch_size and not epsilon <= 0:
            train2(replay_buffer, the_agent, loss_fn, optimizer, scheduler, gamma)
            
        
        cumulative_reward += reward
        
        #6. check max number of time steps has been reached or if game is complete
        if step_count >= environment.max_steps or done:
            return cumulative_reward, video
        
        state = preprocessed_next_state
        
        step_count += 1
        
        

##### Episodic Loop  

In [22]:
def run():
    cfg = vanilla_config()
    with wandb.init(project="BeeGass-Agents", entity="beegass", config=cfg, monitor_gym=True):
        config = wandb.config
        
        # initialize gym environment
        environment = Gym_Env(env_name='ALE/Breakout-v5', max_steps=config.max_steps, max_episodes=config.max_episodes)
        
        # initialize prediction network
        #pred_net = Deep_Q_Network(environment.le_env.action_space.n).to(device)
        pred_net = Deep_Q_Network().to(device)
        
        # initialize agent that contains both prediction network and target network
        the_agent = Agent(pred_model=pred_net)
        
        # define loss function
        loss_fn = nn.MSELoss()
        
        # define optimizer
        optimizer = build_optimizer(model=the_agent.agent, 
                                    optimizer_name='adam', 
                                    learning_rate=config.alpha, 
                                    weight_decay=0.01, 
                                    momentum=0.9)
        
        # define scheduler
        scheduler = build_scheduler(optimizer, 
                                    sched_name='reduce_lr', 
                                    patience=5, 
                                    verbose=True)
        
        # initialize replay buffer
        replay_buffer = Replay_Buffer(capacity=config.replay_buffer_size, mini_batch_size=config.batch_size)
        
        render_flag = False
        epsilon = config.epsilon
        episode_cumulative_reward = 0
        for e in range(environment.max_episodes):
            video = None
            if e + 1 % 1000 == 0:
                render_flag = True
                video = VideoRecorder(environment.le_env, f"./videos/breakout/episode{e}.mp4", enabled=True)
            else:
                render_flag = False
            
            # 0. get initial state, s_{0}, and preprocess it (s_{0} -> preprocessed_s)
            state = environment.le_env.reset()
            
            # 0.1: preprocess state
            preprocessed_state = preprocess(state)
            
            # 1. iterate over steps in episode
            cumulative_reward, video = run_episode(video=video,
                                                render=render_flag,
                                                environment=environment, 
                                                the_agent=the_agent,
                                                replay_buffer=replay_buffer,
                                                state=preprocessed_state, 
                                                epsilon=epsilon,
                                                gamma=config.gamma,
                                                loss_fn=loss_fn,
                                                optimizer=optimizer, 
                                                scheduler=scheduler)
            
            episode_cumulative_reward += cumulative_reward
            wandb.log({"episode": e, "total_reward": episode_cumulative_reward}, step=e)
            wandb.log({"episode": e, "mean_reward": (episode_cumulative_reward/(e+1))}, step=e)
            wandb.log({"episode": e, "reward": cumulative_reward}, step=e)
            
            if render_flag:
                wandb.log({"video": wandb.Video(f"./videos/breakout/episode{e}.mp4", fps=4, format="mp4")})
                video.close
            
            # 3. decay epsilon
            epsilon = config.decay_rate * epsilon
            wandb.log({"episode": e, "epsilon": epsilon}, step=e)

In [23]:
# run()

In [24]:
sweep_id = wandb.sweep(config_tune(), project="dqn-sweeps")

Create sweep with ID: cdyfset2
Sweep URL: https://wandb.ai/beegass/dqn-sweeps/sweeps/cdyfset2


In [25]:
wandb.agent(sweep_id, run, count=5)

wandb: Agent Starting Run: l5xe5z9u with config:
wandb: 	alpha: 0.5046837907491151
wandb: 	batch_size: 128
wandb: 	decay_rate: 0.9536034333773784
wandb: 	epsilon: 0.9514534598466288
wandb: 	gamma: 0.9258062961342872
wandb: 	max_episodes: 500
wandb: 	max_steps: 10000
wandb: 	replay_buffer_size: 10000
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]/home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/home/beegass/.virtualenvs/dl_1/lib/python3.8/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(

wandb: Ctrl + C detected. Stopping sweep.


episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epsilon,█▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_reward,█▅▂▂▂▃▃▃▃▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
reward,██▃█▆▁█▁▆▁▁▁▁▆▁▁▃▃█▁▁▆▁▃▃▁▆▁▁▃▆▁▁▁▃▁▁▁▁▁
total_reward,▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇█████
episode,91
epsilon,0.01203
mean_reward,0.96739
reward,0.0
total_reward,89.0
